Computing the resistance of a triconnected planar graph.  We use the equivalence of resistance networks to
tilings by rectangles observed by @Lisanne.  If we add an edge from the top vertex to the bottom vertex (a "battery" edge) the resulting graph is a biconnected planar graph.  Conversely, corresponding to every biconnected
planar graph is a rectangle dissection.  So it suffices to run through the planar biconnected graphs.

We will compute the rational functions giving the aspect ratio of the rectangle tiled by the smaller rectangles of fixed aspect ratio.

If the graph is just biconnected, then removing two vertices will separate a piece that has a top vertex and bottom vertex and no other connection to the larger graph.  We can replace this by a single resistor having a specified rational function resistance.  If we compute the possible aspect ratios for rectangles tiled by fewer than \(n\) vertices, then we can plug in the precomputed resistances.  So it suffices to consider only 3-connected planar graphs.

Furthermore, as if we take the dual of a graph and invert the resistance of each edge, this is equivalent to rotating the rectangle by \(90^{\circ}\), so it suffices to only consider one of a pair of dual 3-connected planar graphs.

Once we have computed the rational functions representing the aspect ratio of the tiled rectangle, we then compute the real positive roots of numerator-denominator, the values where the tiled rectangle is a square.  Finally, we check that none of the edges end up having zero current.

In [1]:
def equivalent_resistance(graph):
    """
    Compute the rational function describing the resistance of the graph in terms of the resistances of 
    the edges of the graph.
    
    INPUT:
    
    - `graph` The underlying graph.  This graph is assumed to be at least 3-connected and a planar graph with an 
         embedding.  The graph will have one edge labelled "b" (battery) indicating the top and bottom vertex, the
         rest of the edges labelled "r" (resistor)
    """
    n = graph.num_edges() - 1
    
    # set up the underlying field of rational functions
    R = FractionField(PolynomialRing(QQ, 'x', n))
    
    #set up a dictionary of edges and source and sink
    edge_lookup = dict()
    pos = 0
    source = 0
    sink = 0
    for a, b, label in graph.edges(sort=True):
        if label != 'b':
            edge_lookup[(a, b)] = pos
            edge_lookup[(b, a)] = pos
            pos += 1
        else:
            if a < b:
                source = a
                sink = b
            else:
                source = b
                sink = a
    
    vects = []
    
    # flows out of the source
    source_current = [0 for _ in range(n)]
    for v in graph.neighbor_iterator(source):
        if v != sink:
            e = edge_lookup[(source, v)]
            source_current[e] = 1 if v > source else -1
    vects.append(source_current)

    # flows through non source sink vertices
    for v in graph.vertices(sort=True):
        if v != source and v != sink:
            temp_current = [0 for _ in range(n)]
            for v1 in graph.neighbor_iterator(v):
                e = edge_lookup[(v, v1)]
                temp_current[e] = 1 if v1 > v else -1
            vects.append(temp_current)
    
    source_sink_paths = []
    # flows around the faces
    for circuit in graph.faces():
        temp_voltage = [0 for _ in range(n)]
        if (source, sink) in circuit:
            for (v1,v2) in circuit:
                if (v1, v2) != (source, sink):
                    e = edge_lookup[(v1, v2)]
                    temp_voltage[e] = -R.gen(e) if v1 < v2 else R.gen(e)
            source_sink_paths.append(temp_voltage)
        elif (sink, source) in circuit:
            for (v1,v2) in circuit:
                if (v1, v2) != (sink, source):
                    e = edge_lookup[(v1, v2)]
                    temp_voltage[e] = R.gen(e) if v1 < v2 else -R.gen(e)
            source_sink_paths.append(temp_voltage)
        else:
            for (v1, v2) in circuit:
                e = edge_lookup[(v1, v2)]
                temp_voltage[e] = R.gen(e) if v1 < v2 else -R.gen(e)
            vects.append(temp_voltage)
            
    # find the current flows through each resistor if we apply unit current from source to sink.
    m = matrix(vects)
    external_current = vector([1 if x == 0 else 0 for x in range(n)]) 
    internal_current = m.inverse()*external_current

    # the voltage drop is the sum of I*R over a path from sourc to sink
    external_voltage = internal_current.dot_product(vector(source_sink_paths[0]))
    ev_sign = external_voltage.numerator().coefficients()[0].sign()
    ev = ev_sign * external_voltage.numerator() / (ev_sign * external_voltage.denominator())
    return (ev, internal_current)       

In [2]:
# compute the triconnected planar graphs with 12 or fewer edges
# we only keep one of a graph and its dual

from collections import Counter
from collections import defaultdict

base_graphs=[]
labelled_graphs=[]
graph_determinants=defaultdict(list)

for n in range(4,10):
    print(n)
    # generate graphs with n vertices, up to 12 edges that are biconnected with minimal degree 3
    gen = graphs.nauty_geng("{0} 0:12 -C -d3".format(n))
    try:
        for g in gen:
            # check for triconnected and planar
            if g.vertex_connectivity() > 2 and g.is_planar(set_embedding=True):
                gd = g.planar_dual()
                # if we have already stored the dual, ignore this
                if not any(gd.is_isomorphic(g1) for g1 in base_graphs):
                    base_graphs.append(g)
    except ValueError:
        pass
        
    print("number of base graphs = ",len(base_graphs))
    
# Compute the polar nets that can be obtained from the above graphs up to isomorphism.
# Label one edge to be the battery, and check if it is not isomorphic to a previous arrangement
for g in base_graphs:
    h = g.copy()
    for u,v,l in g.edges(sort=False):
        h.set_edge_label(u, v, "r")
    g_edges = g.edges(sort=False)
    u,v,l = g_edges[0]
    g2 = h.copy()
    g2.set_edge_label(u, v, "b")
    temp_list = [(u,v,g2)]
    for u,v,l in g_edges[1:]:
        g2 = h.copy()
        g2.set_edge_label(u,v,"b")
        if not any(g2.is_isomorphic(g3,edge_labels=True) for _,_,g3 in temp_list):
            temp_list.append((u,v,g2))
    labelled_graphs.extend(temp_list)
counts = Counter(g.size() for _,_,g in labelled_graphs)
print(counts)
for _,_,g in labelled_graphs:
    if g.size() <= 13:
        graph_determinants[g.num_edges()-1].append(equivalent_resistance(g))


4
number of base graphs =  1
5
number of base graphs =  3
6
number of base graphs =  9
7
number of base graphs =  16
8
number of base graphs =  16
9
number of base graphs =  16
Counter({12: 52, 11: 11, 10: 8, 8: 2, 9: 2, 6: 1})


In [3]:
from itertools import combinations_with_replacement as cwr, product as itprod

R.<w>=PolynomialRing(ZZ)
K=FractionField(R)

# A list where resistances[n] gives a rational function describing the aspect ratio of the enclosing rectangle
# and a description of the topology of the subrectangles (see construct_biconnected for a description)
resistances = [[],[(K(w),(1,0)), (K(1/w),(-1,0))]]

def construct_biconnected(n):
    """
    Compute all the rational functions that can occur as the aspect ratio of a rectangle made up of
    n rectangles of aspect ratio w.  This assumes that the `resistances` array has been filled out to `n-1`.
    
    This is accomplished by either joining two polar nets in series or parallel. Or else building
    a larger graph by replacing each edge of one of the triconnected graphs with a polar net
    """
    # `s` is the set of possible rational functions
    s = set()
    # The results. Each element of `l` a tuple consisting of 
    #  - The rational function
    #  - The coarse structure
    #     + (1,0) for a single vertical rectangle (-1,0) for a horizontal rectangle.
    #     + (2,0) for 2 subrectangles in series (-2,0) for 2 rectangles in parallel.
    #     + (k,i) for an arrangement scaffolded by a triconnected graph with k edges
    #                 the graph `graph_determinants[k][i]` in particular.
    #     + (-k,i) the triconnected scaffold dual to `graph_deteminants[k][i]`
    # - The fine structure:  A list of `k` edges in the form (j,i) representing `resistances[j][i]`
    l = []
    # compute the series parallel compositions
    for i in range(1,floor(n/2)+1):
        j = n - i
        for i1,r1 in enumerate(resistances[i]):
            for i2,r2 in enumerate(resistances[j]):
                r3 = r1[0] + r2[0]
                if r3 not in s:
                    s.add(r3)
                    l.append((r3,(2,0),[(i,i1),(j,i2)]))
                    s.add(K(1)/r3)
                    l.append((K(1)/r3,(-2,0),[(i,i1),(j,i2)]))
    # Compute the constructions with more complicated triconnected scaffolds
    if n >= 5:
        for comb in range(5,n+1):
            if comb in graph_determinants:
                # run through the different ways of covering the `comb` edges with subrectangles
                for p in iter(Partitions(n,min_length=comb, max_length=comb)):
                    parms = [[[(size,x) for x in pos] for pos in cwr(range(len(resistances[size])), count)]
                            for size, count in enumerate(p.to_exp(), start=1) if count != 0]
                    for xparms in itprod(*parms):
                        # flat parms is a list of subrectangles that can be assigned to the edges
                        flat_parms = tuple(x for y in xparms for x in y)
                        # twist parms is a specific permutation of `flat_parms`
                        #   (`Arrangements` is a list of permutations of a multiset)
                        for twist_parms in Arrangements(flat_parms,comb).list():
                            # `poly_parms` is the list of rational functions for the chosen rectangles
                            poly_parms = [resistances[size][pos][0] for size,pos in twist_parms]
                            for det_index, det in enumerate(graph_determinants[comb]):
                                # func is the rational function representing the aspect ratio.
                                func = det[0](*list(poly_parms))
                                func.reduce()
                                if func not in s:
                                    # check that none of the edges is identically zero
                                    if all([f(*poly_parms) for f in det[1]]):
                                        s.add(func)
                                        l.append((func,(comb,det_index),twist_parms))
                                        s.add(K(1)/func)
                                        l.append((K(1)/func, (-comb,det_index), twist_parms))
    return l
   

def validate_resistances(ratio, graph, edges):
    """
    INPUT:
    - `ratio`  An the aspect ratio of the tiling rectangles as an algebraic number.
    - `graph`  A tuple containing the number of edges and index of the scaffolding graph.
    - `edges`  A list of tuples containing the size and index of the edges of the scaffold.
    
    OUTPUT:
    `True` if none of the small rectangles has size identically zero
    `False` otherwise.
    """
    # series parallel graphs have current flow through all resistors.
    if abs(graph[0]) >= 5:
        # The compute the proportional flow on each edge
        poly_parms = [resistances[size][pos][0](ratio) for size,pos in edges]
        g = graph_determinants[abs(graph[0])][graph[1]]
        # Check that none of the edges have zero flow.
        if not all(f(*poly_parms) for f in g[1]):
            return False
    for size, index in edges:
        # recursively check the smaller rectangles.
        if size >= 5:
            r = resistances[size][index]
            if not validate_resistances(ratio, r[1], r[2]):
                return False
    return True

def count_roots(n):
    """
    INPUT:
    - `n` The number of edges.
    
    OUTPUT:
      A dictionary mapping real positive algebraic numbers such that a square can be tiled by `n` rectangles
      having the algebric aspect ratio to an index in the resistances array describing the arrangement of the
      rectangles.
    """
    # The algebraic numbers rejected because they cause degenrate rectangles.
    rej = set()
    # The algebraic number giving valid soulutions.
    s = dict()
    foundone = False
    for index,r in enumerate(resistances[n]):
        f = r[0].numerator() - r[0].denominator()
        # The real roots where the aspect ratio is one (a square)
        for rt,_ in f.roots(AA):
            if rt not in s and rt >= AA(1):
                #if it is new and greater than or equal to one ...
                if abs(r[1][0]) >= 5:
                    # ... and does not cause a degenerate rectangle
                    if not validate_resistances(rt, r[1], r[2]):
                        if rt >= AA(1):
                            rej.add(rt)
                        continue
                s[rt] = index
    print(n, " ", [x for x in rej if x not in s], " ", len(s))
    return s

max_rectangles = 11
root_collection = [dict(), count_roots(1)]
for i in range(2, max_rectangles):
    l = construct_biconnected(i)
    print(i, " ", len(l)/2)
    resistances.append(l)
    s = count_roots(i)
    root_collection.append(s)
    
print([len(resistances[i])/2 for i in range(1,max_rectangles)]) 

print([len(root_collection[i]) for i in range(1, max_rectangles)])


1   []   1
2   3
2   []   1
3   10
3   []   3
4   38
4   []   11
5   164
5   [1]   51
6   787
6   []   245
7   4258
7   []   1372
8   25767
8   []   8522
9   172504
9   []   58347
10   1258949
10   []   433926
[1, 3, 10, 38, 164, 787, 4258, 25767, 172504, 1258949]
[1, 1, 3, 11, 51, 245, 1372, 8522, 58347, 433926]
